# Speed Estimation using Satellite Imagery
B. Salehi, Y. Zhang and M. Zhong, "Automatic Moving Vehicles Information Extraction From Single-Pass WorldView-2 Imagery," in IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, vol. 5, no. 1, pp. 135-145, Feb. 2012, doi: 10.1109/JSTARS.2012.2183117.

## Imports